In [ ]:
%%capture
import os
from pathlib import Path
import pandas as pd
from dj_notebook import activate
import numpy as np

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from edc_pharmacy.analytics.dataframes import no_stock_for_subjects_df
from datetime import datetime
from edc_registration.models import RegisteredSubject

from edc_appointment.analytics import get_appointment_df
from edc_appointment.constants import NEW_APPT
from edc_pharmacy.models import StockRequest, Allocation, ReceiveItem, OrderItem, Lot

from edc_pharmacy.analytics import get_next_scheduled_visit_for_subjects_df
from meta_rando.models import RandomizationList
from edc_pharmacy.models import Stock
from edc_visit_schedule.models import SubjectScheduleHistory
from django.apps import apps as django_apps
from django.db.models import Count
from django_pandas.io import read_frame
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from edc_pharmacy.models import Container
from great_tables import GT, html, loc, style
from PIL import Image
from edc_pdutils.dataframes.get_subject_visit import convert_visit_code_to_float


In [ ]:
from edc_model_to_dataframe import read_frame_edc
from meta_subject.models import FollowupExamination

df = read_frame_edc(FollowupExamination.objects.all(), drop_sys_columns=True, drop_action_item_columns=True)
df = df.replace("none", pd.NA)
df = df.replace("none", pd.NA)
df = df.fillna(pd.NA)
convert_visit_code_to_float(df)

In [ ]:
from edc_analytics.stata import get_stata_labels_from_model

df = df[["subject_identifier", "subject_visit_id", "report_datetime", "visit_code", "site_id", "site_name", "visit_reason", "symptoms","symptoms_detail", "symptoms_sought_care", "symptoms_g3", "symptoms_g4", "comment"]].copy().reset_index(drop=True)

df = df.astype(
    {col: "Float64" for col in df.select_dtypes(include=["float", "float64"]).columns}
)
df_meds = df.astype(
    {col: "Int64" for col in df.select_dtypes(include=["int", "int64"]).columns}
)
df = df.astype(
    {
        col: "datetime64[ns]"
        for col in df.select_dtypes(include=["datetime", "datetime64"]).columns
    }
)
df = df.astype(
    {
        col: str
        for col in df.select_dtypes(include=["object"]).columns
    }
)
df = df.fillna(pd.NA)

variable_labels = {}
variable_labels.update(**get_stata_labels_from_model(df, model="meta_subject.followupexamination", suffix=None))

df.to_stata(
    path=analysis_folder / "followupexamination.dta",
    variable_labels=variable_labels,
    version=118,
    write_index=False,
)

In [ ]:
df

In [ ]:

def get_great_table(df:pd.DataFrame, title:str, footnote:str|None=None):
    return (GT(df)
        .tab_header(title=html(title))
        .cols_align(align="left", columns=[0])
        .cols_align(align="right", columns=list(range(1, len(df.columns))))
        .opt_stylize(style=5)
        .opt_row_striping(row_striping=False)
        .opt_vertical_padding(scale=1.2)
        .opt_horizontal_padding(scale=1.0)
        .tab_options(
            stub_background_color="white",
            row_group_border_bottom_style="hidden",
            row_group_padding=0.5,
            row_group_background_color="white",
            table_background_color="white",
            table_font_size=12,
        )
        .tab_style(
            style=[style.fill(color="white"), style.text(color="black")],
            locations=loc.body(columns=list(range(len(df.columns))), rows=list(range(0, len(df)))),
        )
        .tab_style(
            style=[style.fill(color="lightgrey"), style.text(color="black")],
            locations=loc.body(columns=list(range(len(df.columns))), rows=[len(df)-1]),
        )
        .tab_source_note(source_note=html(footnote or ""))
        .tab_style(
            style=style.text(color="black", size="small"),
            locations=loc.footer(),
        )


    )


In [ ]:
start_from_appt_date = datetime(2025,5,15)

In [ ]:
# get rando
df_rando = read_frame(RandomizationList.objects.values("subject_identifier", "assignment").filter(subject_identifier__isnull=False))

In [ ]:
# get appointments
df_appt = get_appointment_df()
print(f"{len(df_appt[(df_appt.appt_status==NEW_APPT) & (df_appt.appt_datetime >= start_from_appt_date) & (df_appt.appt_datetime < datetime(2026,3,1)) & (df_appt.visit_code!=1480.0)])} appointments after filtering")

In [ ]:
# create a dataframe of subjects still on the 'schedule' schedule
# use SubjectScheduleHistory where offschedule_datetime is null
df_subject_schedule = read_frame(SubjectScheduleHistory.objects.values("subject_identifier", "visit_schedule_name", "schedule_name", "onschedule_datetime", "offschedule_datetime").filter(offschedule_datetime__isnull=True, schedule_name="schedule"))

print(f"{len(df_subject_schedule)} subjects currently onstudy")

In [ ]:
# for now merge with the unfiltered df_appt
df_main = df_subject_schedule.merge(
    df_appt[["appointment_id", "subject_identifier", "visit_code", "visit_code_str", "appt_datetime", "baseline_datetime", "endline_visit_code", "visit_code_sequence", "appt_status"]],
    on="subject_identifier",
    how="left")
# exclude unscheduled,
df_main = df_main[
    (df_main.visit_code_sequence==0) &
    (df_main.visit_schedule_name=="visit_schedule") &
    (df_main.schedule_name=="schedule") &
    (df_main.visit_code<2000.0) &
    (df_main.appt_status==NEW_APPT)
].copy()
print(f"{len(df_main)} new appointments for subjects on study")


In [ ]:
# number of appointments before extended all subjects out to 48m
df_grouped = df_main[
    (df_main.appt_datetime >= start_from_appt_date) &
    (df_main.appt_datetime < datetime(2026,3,1)) &
    (df_main.visit_code!=1480.0)
].visit_code.value_counts().reset_index(name="appointments").sort_values(by="visit_code", ascending=True).reset_index(drop=True)
df_grouped["cumsum"] = df_grouped.appointments.cumsum()
df_grouped["cumsum"].max()


In [ ]:
df_main

In [ ]:
# now extend everyone to 48 months.
# Subjects are in the process of consenting for extended
# followup. Assume ALL have done so by filling in all
# subject schedules to 48m

# pivot
df_pivot = df_main[
    (df_main.visit_code_sequence==0) &
    (df_main.visit_code<2000.0)
].pivot_table(index="subject_identifier", columns='visit_code', values='appt_datetime', aggfunc='count')
df_pivot.fillna(0, inplace=True)
df_pivot.reset_index(inplace=True)
df_pivot.rename_axis("", axis="columns", inplace=True)

# melt
df_pivot = df_pivot.melt(id_vars="subject_identifier", var_name="visit_code", value_name="exists")
df_pivot["visit_code"] = df_pivot["visit_code"].astype(float)
df_pivot.sort_values(["subject_identifier", "visit_code"], ascending=True, inplace=True)
df_pivot.reset_index(drop=True, inplace=True)

# merge in baseline_datetime
df_baseline = df_appt[df_appt.visit_code==1000.0][["subject_identifier", "baseline_datetime"]]
df_pivot = df_pivot.merge(df_baseline, on=["subject_identifier"], how="left")
df_pivot.reset_index(drop=True, inplace=True)

# merge df_main back in
df_pivot = df_pivot.merge(df_main[["subject_identifier", "visit_code", "appt_datetime", "appt_status"]], on=["subject_identifier","visit_code"], how="left")
df_pivot

In [ ]:
# len(df_pivot[(df_pivot.appt_datetime>=datetime(2025,1,1)) & (df_pivot.visit_code==MONTH48)])/3

In [ ]:
# extend no one!
# df_pivot = df_pivot[df_pivot.exists==1].copy()
# df_pivot.reset_index(drop=True, inplace=True)


In [ ]:
# add appointments do not have an appt_datetime, so calculate
# using the visit schedule relative to baseline_datetime
visit_schedule = site_visit_schedules.get_visit_schedule("visit_schedule")
schedule = visit_schedule.schedules.get("schedule")
mapping = {k: visit.rbase for k,visit in schedule.visits.items()}

def estimate_appt_datetime(row):
    if pd.isna(row["appt_datetime"]):
        row["appt_datetime"] = row["baseline_datetime"] +  mapping.get(str(int(row["visit_code"])))
    return row

df_pivot = df_pivot.apply(estimate_appt_datetime, axis=1)
df_pivot.sort_values(by=["subject_identifier", "visit_code"], ascending=True, inplace=True)
df_pivot.reset_index(drop=True, inplace=True)

# merge in assignment
df_pivot = df_pivot.merge(df_rando, on="subject_identifier", how="left")
df_pivot.reset_index(drop=True, inplace=True)

# flag added appointments as NEW
df_pivot.loc[df_pivot.exists==0.0, "appt_status"] = NEW_APPT

print(f"{len(df_pivot)} appointments")

In [ ]:
# df_subject_appointments is a dataframe of appointments
# - only include NEW appointments
# - only include appts between today (2025,4,4) and before (2026,3,1).
# - exclude the last visit (48m) since no meds are dispensed then.
cutoff_date = datetime(2026,3,1)
df_subject_appointments = df_pivot[
    (df_pivot.appt_status==NEW_APPT) &
    (df_pivot.appt_datetime >= start_from_appt_date) &
    (df_pivot.appt_datetime < cutoff_date) &
    (df_pivot.visit_code!=1480.0)
].copy()
print(f"{len(df_subject_appointments)} appointments")

In [ ]:
n = df_subject_appointments.subject_identifier.nunique()
print(f"{n} subjects")


In [ ]:
(len(df_subject_appointments[df_subject_appointments.appt_datetime>=datetime(2026,1,1)])/36)/5

In [ ]:
# summarize the appointments
df_summary = df_subject_appointments.visit_code.value_counts().reset_index(name="appointments").sort_values(by=["visit_code"], ascending=True)
df_summary["cumsum"] = df_summary.appointments.cumsum()
df_summary

In [ ]:
df = df_subject_appointments.assignment.value_counts(dropna=False).reset_index()
df.rename(columns={"count":"appointments"}, inplace=True)
df["bottles"] = df.appointments * 3
df["tablets"] = df.bottles * 128

# we need this many bottles / tablets by assignment
# filter
df.loc[len(df)] = {"appointments": df.appointments.sum(), "bottles": df.bottles.sum(), "tablets": df.tablets.sum()}
df

In [ ]:
gt = get_great_table(
    df,
    "Table 1: IMP Bottles of 128 needed<BR><small>as of 2025-05-15</small>",
    footnote=(
        "<ol>"
        "<li>assume all participants consent for extended followup."
        "<li>Need 3 bottles every three months"
        "<li>48m appointment is excluded"
        "<li>Only prepare for appointments scheduled before 2026-03-01."
        "</ol>"
    ))
gt.show()

In [ ]:

# save as png
gt.save(analysis_folder / "pharmacy_tbl1.png")
# export to PDF
image = Image.open(analysis_folder / "pharmacy_tbl1.png")
image = image.resize((image.width * 6, image.height * 6), Image.LANCZOS)
image.save(analysis_folder / "pharmacy_tbl1.pdf", "PDF", resolution=800, optimize=True, quality=95)

In [ ]:
# now lets look at the stock
df_stock = read_frame(Stock.objects.values("code", "lot_id", "container__name", "confirmation", "allocation", "dispense", "qty_in", "qty_out", "unit_qty_in", "unit_qty_out").all(), verbose=False)
df_stock = df_stock.fillna(pd.NA)

# merge in assignment
df_lot = read_frame(Lot.objects.values("id", "assignment__name").all(), verbose=False)
df_lot.rename(columns={"id":"lot_id", "assignment__name": "assignment"}, inplace=True)
df_stock = df_stock.merge(df_lot[["lot_id", "assignment"]], on="lot_id", how="left")
df_stock.rename(columns={"container__name":"container"}, inplace=True)
df_stock.reset_index(drop=True, inplace=True)

In [ ]:
# merge in container columns
df_container = read_frame(Container.objects.all())
df_container.rename(columns={"name": "container", "display_name": "container_display_name", "units": "container_units", "qty": "container_qty"}, inplace=True)
df_stock = df_stock.merge(df_container[["container", "container_display_name", "container_type", "container_units", "container_qty"]], on="container", how="left")
df_stock.reset_index(drop=True, inplace=True)

# calculate bal
df_stock["bal"] = df_stock["unit_qty_in"] - df_stock["unit_qty_out"]


In [ ]:
# show the balance of tablets decanted to bottles by assignment (on the EDC)
df2 = df_stock[df_stock.container_display_name=="Bottle 128"].groupby(by=["assignment"]).bal.agg("sum").reset_index()
df2.loc[len(df2)] = {"bal": df2.bal.sum()}
df2

In [ ]:
# some bottles, as of today, have not been captured in the system
# here is an estimate of what has been decanted into bottles but not labelled.
# in the system, these tablets would appear on the EDC as still in buckets
df3 = df2.copy()
df3 = df3.drop(len(df3) - 1)
placebo_unlabelled = 0 # 21*128*128
active_unlabelled = 0 # 25*191*128

# adding in the estimates, this is about what we have bottled
df3.loc[df3.assignment=="placebo", "bal"] +=  placebo_unlabelled
df3.loc[df3.assignment=="active", "bal"] +=  active_unlabelled
df3.loc[len(df3)] = {"bal": df3.bal.sum()}
df3

In [ ]:
gt = get_great_table(
    df3,
    "Table 2: IMP tablets in stock<BR><small>as of 2025-04-04</small>",
    # footnote="Includes recently decanted but unlabelled bottles"
    )
gt.show()

In [ ]:
# save as png
gt.save(analysis_folder / "pharmacy_tbl2.png")
# export to PDF
image = Image.open(analysis_folder / "pharmacy_tbl2.png")
image = image.resize((image.width * 6, image.height * 6), Image.LANCZOS)
image.save(analysis_folder / "pharmacy_tbl2.pdf", "PDF", resolution=800, optimize=True, quality=95)

In [ ]:
# tablets: ordered
df_orderitems = read_frame(OrderItem.objects.all())
df_orderitems.qty.sum()

In [ ]:
# tablets: received
df_received_items = read_frame(ReceiveItem.objects.all())
df_received_items.unit_qty.sum()

In [ ]:
# tablets: received into stock
df_stock[df_stock.container_type=="bucket"].unit_qty_in.sum()

In [ ]:
# tablets: decanted from buckets into bottles
df_stock[df_stock.container_type=="bucket"].unit_qty_out.sum()

In [ ]:
# tablets: total in bottles
df_stock[df_stock.container_type=="Bottle"].unit_qty_in.sum()

In [ ]:
# tablets: total bottles available / not yet dispensed BY ASSIGNMENT
# the total matches the total above for column "bal"
df4 = df_stock[(df_stock.container_type=="Bottle") & ~(df_stock.confirmation.isna()) & ~(df_stock.dispense.isna())].groupby(by=["assignment"]).unit_qty_in.sum().reset_index()
df4["subtotal"] = np.nan
df4.loc[len(df4)] = {"subtotal": df4.unit_qty_in.sum()}
df["dispensed"] = True

df5 = df_stock[(df_stock.container_type=="Bottle") & ~(df_stock.confirmation.isna()) & (df_stock.dispense.isna())].groupby(by=["assignment"]).unit_qty_in.sum().reset_index()
df5.loc[df5.assignment=="placebo", "unit_qty_in"] +=  placebo_unlabelled
df5.loc[df5.assignment=="active", "unit_qty_in"] +=  active_unlabelled
df5["subtotal"] = np.nan
df5.loc[len(df5)] = {"subtotal" : df5.unit_qty_in.sum()}
df5["dispensed"] = False

df6 = pd.concat([df4, df5])
df6["total"] = np.nan
df6.reset_index(drop=True, inplace=True)
df6.loc[len(df6)] = {"total": df6.subtotal.sum()}
df6 = df6[["dispensed", "assignment", "unit_qty_in", "subtotal", "total"]]
df6

In [ ]:
from meta_visit_schedule.constants import MONTH36

df_appt[(df_appt.visit_code_str==MONTH36) & (df_appt.appt_datetime >= datetime(2024,12,15)) & (df_appt.appt_status==NEW_APPT) & (df_appt.appt_datetime <= datetime(2026,2,28))]

In [ ]:
def remove_subjects_where_stock_on_site(stock_request: StockRequest, df: pd.DataFrame):
    stock_model_cls = django_apps.get_model("edc_pharmacy.Stock")
    qs_stock = (
        stock_model_cls.objects.values(
            "allocation__registered_subject__subject_identifier", "code"
        )
        .filter(location=stock_request.location, qty=1)
        .annotate(count=Count("allocation__registered_subject__subject_identifier"))
    )
    df_stock = read_frame(qs_stock)
    df_stock = df_stock.rename(
        columns={
            "allocation__registered_subject__subject_identifier": "subject_identifier",
            "count": "stock_qty",
        }
    )
    if not df.empty and not df_stock.empty:
        df_subject = df.copy()
        df_subject["code"] = None
        df = df.merge(df_stock, on="subject_identifier", how="left")
        for index, row in df.iterrows():
            qty_needed = stock_request.containers_per_subject - len(df[df.subject_identifier == row.subject_identifier])
            if qty_needed > 0:
               for _ in range(0, qty_needed):
                   df = pd.concat([df, df_subject])
    else:
        df["code"] = None
    df["stock_qty"] = 0.0
    df = df.reset_index(drop=True)
    return df


In [ ]:
def pad_with_null_rows(df, qty_needed):
    padded_data = []
    for index, row in df.iterrows():
        customer = row['subject']
        products = row['product_code']
        # Pad the products list with None to make its length x
        products += [None] * (qty_needed - len(products))
        # Create x rows for each customer
        for product in products:
            padded_data.append({'customer': customer, 'product_code': product})
    return pd.DataFrame(padded_data)

In [ ]:
pk = "5455cf66-b8e5-449c-a1e8-24d3325026d7"
stock_request = StockRequest.objects.get(pk=pk)


In [ ]:
df_subjects = get_next_scheduled_visit_for_subjects_df(stock_request)
df_subjects

In [ ]:
df = df_subjects.copy()
stock_model_cls = django_apps.get_model("edc_pharmacy.Stock")
qs_stock = (
    stock_model_cls.objects.values(
        "allocation__registered_subject__subject_identifier", "code"
    )
    .filter(location=stock_request.location, qty=1)
    .annotate(count=Count("allocation__registered_subject__subject_identifier"))
)
df_stock = read_frame(qs_stock)
df_stock = df_stock.rename(
    columns={
        "allocation__registered_subject__subject_identifier": "subject_identifier",
        "count": "stock_qty",
    }
)
df_stock

In [ ]:
df.merge(df_stock, on="subject_identifier", how="left")

In [ ]:
if not df.empty and not df_stock.empty:
    df_subject = df.copy()
    df_subject["code"] = None
    df = df.merge(df_stock, on="subject_identifier", how="left")
    for index, row in df.iterrows():
        qty_needed = stock_request.containers_per_subject - len(df[df.subject_identifier == row.subject_identifier])
        if qty_needed > 0:
           for _ in range(0, qty_needed):
               df = pd.concat([df, df_subject])
else:
    df["code"] = None
df["stock_qty"] = 0.0
df = df.reset_index(drop=True)
df

In [ ]:
df.loc[df.index.repeat(3)]

In [ ]:
if not df.empty and not df_stock.empty:
    df = df.merge(df_stock, on="subject_identifier", how="left")
else:
    df["code"] = None
df["stock_qty"] = 0.0
df = df.reset_index(drop=True)
df

In [ ]:
df = remove_subjects_where_stock_on_site(stock_request, df_subjects)
df

In [ ]:
df_instock = df[~df.code.isna()]
df_instock = df_instock.reset_index(drop=True)
df_instock = df_instock.sort_values(by=["subject_identifier"])

df_nostock = df[df.code.isna()]
df_nostock = df_nostock.reset_index(drop=True)
df_nostock = df_nostock.loc[
    df_nostock.index.repeat(stock_request.containers_per_subject)
].reset_index(drop=True)
df_nostock = df_nostock.sort_values(by=["subject_identifier"])
df_nostock["code"] = df_nostock["code"].fillna("---")


In [ ]:
no_stock_for_subjects_df()

In [ ]:
df_schedule = read_frame(SubjectScheduleHistory.objects.values("subject_identifier", "visit_schedule_name","schedule_name", "offschedule_datetime").all())


In [ ]:
df_schedule = df_schedule[(df_schedule.visit_schedule_name=="visit_schedule") & (df_schedule.schedule_name=="schedule")  & df_schedule.offschedule_datetime.isna()]
df_schedule.reset_index(drop=True, inplace=True)

In [ ]:
df_stock = read_frame(Stock.objects.all(), verbose=False)
df_stock_on_site = df_stock[(df_stock.confirmed_at_site==True) & (df_stock.dispensed==False)].copy()
df_stock_on_site.reset_index(drop=True, inplace=True)
df_stock_on_site = df_stock_on_site.drop(columns=["subject_identifier"])


In [ ]:
df_allocation =  read_frame(Allocation.objects.values("id", "registered_subject").all(), verbose=False)
df_rs = read_frame(RegisteredSubject.objects.values("id", "subject_identifier").all(), verbose=False)
df_allocation = df_allocation.merge(df_rs[["id", "subject_identifier"]], how="left", left_on="registered_subject", right_on="id", suffixes=["_allocation", "_rs"])

In [ ]:
df_stock_on_site = df_stock_on_site.merge(df_allocation[["id_allocation", "subject_identifier"]], how="left", left_on="allocation", right_on="id_allocation")

In [ ]:
df = pd.merge(df_schedule[["subject_identifier", 'offschedule_datetime']], df_stock_on_site, on="subject_identifier", how="left")
df= df[df.code.isna()][["subject_identifier", ]].sort_values(by=["subject_identifier"]).reset_index(drop=True)

In [ ]:
df_appt = get_next_scheduled_visit_for_subjects_df()
df_appt = df_appt[["subject_identifier", "site_id", "visit_code", "appt_datetime", "baseline_datetime"]].copy()
df_appt.reset_index(drop=True, inplace=True)

In [ ]:

df = df.merge(df_appt, how="left", on="subject_identifier")
df = df[(df.appt_datetime.notna())]
df.reset_index(drop=True, inplace=True)

In [ ]:
utc_now = pd.Timestamp.utcnow().tz_localize(None)
df["relative_days"] = (df.appt_datetime - utc_now).dt.days
df_final = df[(df.relative_days >= -105)].copy()
df_final.reset_index(drop=True, inplace=True)
df_final

In [ ]:
RegisteredSubject.objects.filter(site_id=10)